# Create a Ryu OpenFlow Controller

In [ ]:
import json
import os
import chi

#Config with your project and site
chi.set('project_name', 'chameleon')   # Replace with your project name
chi.set('region_name', 'CHI@UC')       # Optional, defaults to 'CHI@UC'
chi.use_site('CHI@UC')                 # Authenticate to CHI@UC site

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_RyuControllerTutorial_"
server_name = prefix+'Server'
lease_name = username+'Lease'

#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_skylake"
server_count=1
network_name='sharednet1'

In [ ]:
import chi.lease
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name,
                               reservations=reservation_list,
                               start_date=start_date,
                               end_date=end_date)
#Print the lease info
print(json.dumps(lease, indent=2))

In [ ]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

In [ ]:
import chi.server

# Create the server
server = chi.server.create_server(server_name,
                                  reservation_id=compute_reservation_id,
                                  network_name=network_name,
                                  image_name=image_name)
# Wait until the server is active
chi.server.wait_for_active(server.id)

In [ ]:
floating_ip = chi.server.associate_floating_ip(server.id)

print(f'Floating IP: {floating_ip}')

In [ ]:
ryu_app = 'simple_switch_13_custom_chameleon.py'
ryu_port = '6653'
mirror_port = ''

script= '#!/bin/bash'   '\n' \
    '{'   '\n' \
    'yum install -y yum-utils device-mapper-persistent-data lvm2 vim'   '\n' \
    'yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo'   '\n' \
    'yum install -y docker-ce'   '\n' \
    'systemctl start docker'   '\n' \
    '   \n' \
    r'RECIPE_REPO=\"https://github.com/RENCI-NRIG/exogeni-recipes.git\"'   '\n' \
    'RECIPE_DIR=\"/opt/exogeni-recipes\"'   '\n' \
    'RECIPE_APP=\"openflow-controller/docker\"'   '\n' \
    'DOCKER_IMAGE=\"centos-ryu\"'   '\n' \
    'DOCKER_CONTAINER_NAME=\"ryu-controller\"'   '\n' \
    'OFP_TCP_LISTEN_PORT=\"'+ryu_port+'\"'   '\n' \
    'RYU_APP=\"/opt/ryu_app/'+ryu_app+'\"'   '\n' \
    'MIRROR_PORT='+mirror_port+   '\n' \
    '   \n' \
    'git clone  --no-checkout \${RECIPE_REPO} \${RECIPE_DIR}'   '\n' \
    'cd \${RECIPE_DIR} && git config core.sparsecheckout true'   '\n' \
    'echo \"\${RECIPE_APP}/*\" >> .git/info/sparse-checkout'   '\n' \
    'git read-tree -m -u HEAD'   '\n' \
    '   \n' \
    'cd \${RECIPE_DIR}/\${RECIPE_APP}'   '\n' \
    'docker volume create var_run_ryu'   '\n' \
    'docker volume create var_log_ryu'   '\n' \
    'docker volume create opt_ryu'   '\n' \
    'docker volume create opt_ryu_chameleon'   '\n' \
    "sed -r -i \'s/^(RYU_APP=.*)/#\1/g\' ryu_start.sh"   '\n' \
    "sed -r -i \'s/^(OFP_TCP_LISTEN_PORT=.*)/#\1/g\' ryu_start.sh"   '\n' \
    '   \n' \
    'docker build -t \${DOCKER_IMAGE} .'   '\n' \
    '   \n' \
    'docker run --rm -dit -p \${OFP_TCP_LISTEN_PORT}:\${OFP_TCP_LISTEN_PORT} -p 8080:8080 -v opt_ryu_chameleon:/opt/ryu_chameleon -v opt_ryu:/opt/ryu -v var_log_ryu:/var/log/ryu -v var_run_ryu:/var/run/ryu -e RYU_APP=\${RYU_APP} -e OFP_TCP_LISTEN_PORT=\${OFP_TCP_LISTEN_PORT}  --name=\${DOCKER_CONTAINER_NAME} \${DOCKER_IMAGE}'   '\n' \
    '   \n' \
    'echo done'   '\n' \
    '   \n' \
    '} > /tmp/boot.log 2>&1'   '\n'

In [ ]:
import paramiko
#from scp import SCPClient, SCPException

key = paramiko.RSAKey.from_private_key_file("/home/pruth/work/pruth-jupyter")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(floating_ip, username='cc', pkey=key)

stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
print(stdout.read())
print(stderr.read())

client.close()
print('done!')

In [ ]:
chi.server.delete_server(server.id)
chi.lease.delete_lease(lease['id'])